In [2]:
!pip install torch transformers

  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl (766.7 MB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)


In [3]:
import boto3
import pandas as pd
from io import StringIO
from transformers import pipeline

# Load the pre-trained sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis")

# Configure AWS S3 client
s3 = boto3.client('s3')
bucket = "zanggar"  # Specify your bucket name
prefix = "technodom_reviews/cleaned_final_output/"  # Specify your prefix (path)

# Get the list of CSV files from S3
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
dataframes = []

for obj in response.get("Contents", []):
    key = obj["Key"]
    if key.endswith(".csv"):
        print(f"Downloading: {key}")
        # Get the content of the CSV file from S3
        obj_data = s3.get_object(Bucket=bucket, Key=key)
        content = obj_data["Body"].read().decode("utf-8")
        
        # Read the CSV content into a DataFrame
        df = pd.read_csv(StringIO(content), on_bad_lines="skip")
        
        if "translated" in df.columns:
            # Keep only the translated reviews
            df = df[["translated"]].dropna()

            # Apply sentiment analysis to each review
            df["label"] = df["translated"].apply(lambda text: sentiment_analyzer(str(text)[:512])[0]["label"].lower())
            
            # Add the DataFrame to the list
            dataframes.append(df)

# Combine all dataframes into one
final_df = pd.concat(dataframes)

# Save the results to a CSV file
final_df.to_csv('sentiment_labels.csv', index=False)

# Display the first few rows of the final DataFrame for verification
print(final_df.head())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Downloading: technodom_reviews/cleaned_final_output/part-00000-f8582dcd-65de-46a8-a8bc-e98410eeb8c4-c000.csv
                                          translated     label
0          Good phone, I am happy with the purchase.  positive
1               It works well, does not even freeze)  positive
2  For some reason, they did not give a gift box ...  negative
3                                     Only the price  negative
4                                        Excellent 👌  positive


In [4]:
!aws s3 cp sentiment_labels.csv s3://zanggar/technodom_reviews/results/sentiment_labels.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


upload: ./sentiment_labels.csv to s3://zanggar/technodom_reviews/results/sentiment_labels.csv
